In [10]:
import datetime
import boto3
from keras.models import load_model
import cv2
import numpy as np
import pygame
from twilio.rest import Client
import time
import tkinter as tk
from PIL import Image, ImageTk

# Initialize pygame
pygame.mixer.init()

# Twilio credentials
account_sid = 'AC81c772ff2e061bf7679cbb6432bea2fc'
auth_token = 'd8dd1c8580bd209daabee899fde02088'
from_phone_number = '+12513134943'
to_phone_number = '+919145709780'

# Initialize Twilio client
client = Client(account_sid, auth_token)

# Load the model
model = load_model("keras_Model.h5", compile=False)

# Load the labels
class_names = open("labels.txt", "r").readlines()

# CAMERA can be 0 or 1 based on the default camera of your computer
camera = cv2.VideoCapture(0)

# Get the dimensions of the webcam feed
width = int(camera.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' codec for MP4 format
output_filename = datetime.datetime.now().strftime("output_%Y%m%d%H%M%S.mp4")
out = cv2.VideoWriter(output_filename, fourcc, 20.0, (width, height))

# Initialize the previous class index to track changes
prev_class_index = None

# Confidence threshold
confidence_threshold = 0.9

# Flags to check if the beep sound and SMS should be triggered
play_beep = False
send_message = False

# Initialize variables for framerate calculation
start_time = time.time()
frame_count = 0

# Function to detect objects
def detect_objects():
    global prev_class_index, play_beep, send_message
    play_beep = False
    send_message = False

    # Grab the web camera's image
    ret, image = camera.read()

    # Resize the raw image into (224-height,224-width) pixels
    image_resized = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)

    # Write the frame to the video file
    out.write(image)

    # Make the image a numpy array and reshape it to the model's input shape
    image_norm = np.asarray(image_resized, dtype=np.float32).reshape(1, 224, 224, 3)

    # Normalize the image array
    image_norm = (image_norm / 127.5) - 1

    # Prediction of the model
    prediction = model.predict(image_norm)
    index = np.argmax(prediction)
    class_name = class_names[index]
    confidence_score = prediction[0][index]

    # Print prediction and confidence score
    print("Class:", class_name[2:], end="")
    print("Confidence Score:", str(np.round(confidence_score * 100))[:-2], "%")

    # Check if the predicted class is "fire" or "weapon" and confidence score is above the threshold
    if confidence_score > confidence_threshold and class_name[2:].strip().lower() in ["fire", "weapon"]:
        play_beep = True
        send_message = True

    # Beep sound if the predicted class changes randomly and play_beep flag is True
    if prev_class_index is not None and prev_class_index != index and play_beep:
        pygame.mixer.music.load("beep.wav")  # Replace with your sound file
        pygame.mixer.music.play()

    # Send a message if the predicted class changes and send_message flag is True
    if prev_class_index is not None and prev_class_index != index and send_message:
        message = client.messages.create(
            body=f"Fire or Weapon detected! Class: {class_name[2:]}",
            from_=from_phone_number,
            to=to_phone_number
        )
        print(f"Message sent: {message.sid}")

    prev_class_index = index

    # Framerate calculation
    global frame_count, start_time
    frame_count += 1
    elapsed_time = time.time() - start_time
    if elapsed_time >= 1.0:  # Update framerate every second
        framerate = frame_count / elapsed_time
        print(f"Framerate: {framerate:.2f} FPS")
        start_time = time.time()
        frame_count = 0

    # Display the webcam feed
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)
    image = ImageTk.PhotoImage(image=image)
    canvas.image = image  # Keep a reference to prevent garbage collection
    canvas.create_image(0, 0, anchor=tk.NW, image=image)

    # Schedule the next detection
    window.after(10, detect_objects)

# Function to handle closing the window
def on_closing():
    camera.release()
    out.release()
    cv2.destroyAllWindows()
    window.destroy()

# Function to quit the application
def quit_application():
    on_closing()

# Function to validate login credentials
def validate_login():
    global window
    username = entry_username.get()
    password = entry_password.get()

    if username == "admin" and password == "1234":
        window_login.destroy()
        window = tk.Tk()
        window.title("TKIET SECURITY SYSTEM")

        # Set background image
        img = Image.open("img.png")
        bg_image = ImageTk.PhotoImage(img)
        bg_label = tk.Label(window, image=bg_image)
        bg_label.image = bg_image  # Keep a reference to the image
        bg_label.place(relwidth=1, relheight=1)

        # Create a label for the title
        lbl_title = tk.Label(window, text="TKIET SECURITY SYSTEM", font=('Arial', 24))
        lbl_title.pack(pady=20)

        # Create a Canvas widget to display the webcam feed
        global canvas
        canvas = tk.Canvas(window, width=width, height=height)
        canvas.pack()

        # Button to trigger object detection
        btn_detect = tk.Button(window, text="Detect Objects", command=detect_objects)
        btn_detect.pack(pady=10)

        # Button to quit the application
        btn_quit = tk.Button(window, text="Quit", command=quit_application)
        btn_quit.pack(pady=5)

        window.after(10, detect_objects)
    else:
        lbl_login_status.config(text="Invalid username or password", fg="red")

# Create a Tkinter window for login
window_login = tk.Tk()
window_login.title("Login")

# Set background image
img = Image.open("img.png")
bg_image = ImageTk.PhotoImage(img)
bg_label = tk.Label(window_login, image=bg_image)
bg_label.image = bg_image  # Keep a reference to the image
bg_label.place(relwidth=1, relheight=1)

# Username label and entry
lbl_username = tk.Label(window_login, text="Username:")
lbl_username.pack(pady=5)
entry_username = tk.Entry(window_login)
entry_username.pack(pady=5)

# Password label and entry
lbl_password = tk.Label(window_login, text="Password:")
lbl_password.pack(pady=5)
entry_password = tk.Entry(window_login, show="*")
entry_password.pack(pady=5)

# Login button
btn_login = tk.Button(window_login, text="Login", command=validate_login)
btn_login.pack(pady=5)

# Login status label
lbl_login_status = tk.Label(window_login, text="", fg="red")
lbl_login_status.pack(pady=5)

# Run the Tkinter event loop for login window
window_login.mainloop()


TclError: image "pyimage9" doesn't exist